This notebook introduces items for U.S. States into the knowledgebase. In this case, we use a longstanding "authority" for information on the administrative/political divisions in the U.S. - the U.S. Census Bureau's TIGER data system. It provides a somewhat superior base to build from to going directly to Wikidata as a starting point. The [TIGER web system](https://tigerweb.geo.census.gov/) advertises several different routes to data. While the "REST services" might be an obvious choice, they suffer from what many GIS-based data systems have as a weakness: they are built and organized for use in a GIS context and not for other types of data access and use. At some level, we may care about GIS services in running knowledge organization processing, but our immediate use cases have information that is already "indexed" to things like place names and identifiers. So, we need can use a simpler source and come back to the GIS-focused services later if needed.

The "Data Files" listings from TIGER provide nation, state, and county level listings of the political/administrative division information we care about for this knowledgebase. They are in simple HTML format tables, which are simple enough to read and process using a library like Pandas.

For data source documentation, I'm following the same approach as I used for countries - laying out items in the Wikibase instance to link to as references from resulting claims and to dynamically drive as much of the process as makes sense. This use case introduces a different property to house URLs pointing to HTML tables along with an ordinal value indicating which table contain the data in question (only one table in this case, but we could have other situations).

In [1]:
import os
import requests
import pandas as pd
from wbmaker import WikibaseConnection
from wikibaseintegrator import models, datatypes

eew = WikibaseConnection('eew')

In [2]:
properties = eew.properties()
classes = eew.classification()
datasources = eew.datasources()

In [3]:
display(properties)
display(classes)
display(datasources)

{'instance of': 'P1',
 'subclass of': 'P2',
 'related wikidata property': 'P3',
 'related wikidata item': 'P4',
 'formatter URL': 'P5',
 'data source': 'P6',
 'data format': 'P7',
 'item of this property': 'P8',
 'equivalent property': 'P9',
 'reference URL': 'P10',
 'coordinate location': 'P11',
 'geographical location': 'P12',
 'country': 'P13',
 'U.S. state': 'P14',
 'U.S. county': 'P15',
 'municipality': 'P16',
 'ISO 3166-1 numeric code': 'P17',
 'ISO 3166-1 alpha-2 code': 'P18',
 'ISO 3166-1 alpha-3 code': 'P19',
 'ISO 3166-2 code': 'P20',
 'FIPS 6-4': 'P21',
 'FIPS 5-2 numeric': 'P22',
 'FIPS 5-2 alpha': 'P23',
 'FIPS 55-3': 'P24',
 'GNIS ID': 'P25',
 'NAICS Code': 'P26',
 'SIC code': 'P28',
 'query string': 'P29',
 'caveat': 'P30',
 'html table': 'P31',
 'table ordinal': 'P32',
 'technical documentation': 'P33',
 'FIPS 10-4': 'P34'}

{'property': 'Q2',
 'spatio-temporal entity': 'Q3',
 'location': 'Q4',
 'object': 'Q8',
 'temporal entity': 'Q5',
 'spatial entity': 'Q6',
 'human activity': 'Q7',
 'country': 'Q14',
 'U.S. Territory': 'Q266',
 'U.S. State': 'Q267',
 'U.S. federal district': 'Q320',
 'artificial entity': 'Q9',
 'work': 'Q10',
 'dataset': 'Q11'}

,ds,dsLabel,query_string,html_table
0,https://eew-edgi.wikibase.cloud/entity/Q13,Wikidata listing of world countries,https://query.wikidata.org/sparql?query=SELECT...,None
1,https://eew-edgi.wikibase.cloud/entity/Q264,TIGER data file source for U.S. States,None,https://tigerweb.geo.census.gov/tigerwebmain/F...
2,https://eew-edgi.wikibase.cloud/entity/Q265,Wikidata listing of U.S. states and territories,https://query.wikidata.org/sparql?query=SELECT...,None


# TIGER Source Data

In [4]:
tiger_states_datasource = datasources[datasources.dsLabel == 'TIGER data file source for U.S. States'].iloc[0]
tiger_states_datasource_qid = tiger_states_datasource.ds.split('/')[-1]
tiger_states = eew.get_html_table(tiger_states_datasource.html_table)
tiger_states.head(3)

,MTFCC,OID,GEOID,STATE,STATENS,BASENAME,NAME,LSADC,FUNCSTAT,AREALAND,AREAWATER,UR,REGION,DIVISION,STUSAB,CENTLAT,CENTLON,INTPTLAT,INTPTLON
0,G4000,2749065038753,01,01,01779775,Alabama,Alabama,00,A,131185039975,4582335766,NaN,3,6,AL,+32.7436859,-086.8467951,+32.7395785,-086.8434469
1,G4000,2749045059742,02,02,01785533,Alaska,Alaska,00,A,1478942892154,245378380543,NaN,4,9,AK,+63.4130518,-152.8796317,+63.3473560,-152.8397334
2,G4000,274901053753069,04,04,01779777,Arizona,Arizona,00,A,294365830107,854014485,NaN,4,8,AZ,+34.2751017,-111.6605337,+34.2039362,-111.6063449


### TIGER Data Transformations

We do a couple of things here to generate conformant codes compatible with our target schema and narrow down to those properties we will be using.

In [5]:
usable_tiger_data = []

for index, row in tiger_states.iterrows():
    record_identifier = f"US-{row.STUSAB}"
    if record_identifier == 'DC':
        classification = classes['U.S. federal district']
    else:
        if row.REGION == '9':
            classification = classes['U.S. Territory']
        else:
            classification = classes['U.S. State']

    usable_tiger_data.append({
        'identifier': record_identifier,
        'property': 'label',
        'value': row['NAME'],
        'reference_qid': tiger_states_datasource_qid
    })

    usable_tiger_data.append({
        'identifier': record_identifier,
        'property': properties['instance of'],
        'value': classification,
        'reference_qid': tiger_states_datasource_qid
    })
    
    usable_tiger_data.append({
        'identifier': record_identifier,
        'property': properties['FIPS 5-2 alpha'],
        'value': row.STUSAB,
        'reference_qid': tiger_states_datasource_qid
    })
    
    usable_tiger_data.append({
        'identifier': record_identifier,
        'property': properties['FIPS 5-2 numeric'],
        'value': row.GEOID,
        'reference_qid': tiger_states_datasource_qid
    })
    
    usable_tiger_data.append({
        'identifier': record_identifier,
        'property': properties['ISO 3166-2 code'],
        'value': f"US-{row.STUSAB}",
        'reference_qid': tiger_states_datasource_qid
    })
    
    usable_tiger_data.append({
        'identifier': record_identifier,
        'property': properties['FIPS 10-4'],
        'value': f"US{row.GEOID}",
        'reference_qid': tiger_states_datasource_qid
    })

    usable_tiger_data.append({
        'identifier': record_identifier,
        'property': properties['GNIS ID'],
        'value': row.STATENS.lstrip('0'),
        'reference_qid': tiger_states_datasource_qid
    })

    usable_tiger_data.append({
        'identifier': record_identifier,
        'property': properties['coordinate location'],
        'value': ",".join([str(float(row.CENTLAT)),str(float(row.CENTLON))]),
        'reference_qid': tiger_states_datasource_qid
    })

props_tiger = pd.DataFrame(usable_tiger_data)

# Wikidata Source Data

There are some potentially useful characteristics we could pull from Wikidata for the work in this knowledgebase context. We may also be able to exploit some of the external identifiers assembled for those representations. At the moment, we don't need much but will use the descriptions as something reasonable and record the Wikidaia item QID for later reference. We use a dataset item with a recorded SPARQL query to pull the ISO 3166-2 code that we can use to establish the linkage.

In [6]:
wd_states_datasource = datasources[datasources.dsLabel == 'Wikidata listing of U.S. states and territories'].iloc[0]
wd_states_datasource_qid = wd_states_datasource.ds.split('/')[-1]

sparql_endpoint, sparql_query = eew.parse_sparql_url(wd_states_datasource.query_string)

wd_states = eew.sparql_query(
    endpoint=sparql_endpoint,
    query=sparql_query,
    output='dataframe'
)

usable_wd_records = wd_states.drop_duplicates(subset="iso3166_2", keep="first")[["st","stDescription","iso3166_2"]]
usable_wd_records = usable_wd_records[usable_wd_records.iso3166_2.isin(props_tiger.identifier)].convert_dtypes()

usable_wd_props = []
for index, row in usable_wd_records.iterrows():
    record_identifier = row.iso3166_2
    
    usable_wd_props.append({
        'identifier': record_identifier,
        'property': 'description',
        'value': row.stDescription,
        'reference_qid': wd_states_datasource_qid
    })

    usable_wd_props.append({
        'identifier': record_identifier,
        'property': properties['related wikidata item'],
        'value': row.st.split('/')[-1],
        'reference_qid': wd_states_datasource_qid
    })

props_wd = pd.DataFrame(usable_wd_props)

# Combine TIGER with Select Wikidata Props


In [7]:
tiger_plus_wd = pd.concat([props_tiger, props_wd])
tiger_plus_wd

,identifier,property,value,reference_qid
0,US-AL,label,Alabama,Q264
1,US-AL,P1,Q267,Q264
2,US-AL,P23,AL,Q264
3,US-AL,P22,01,Q264
4,US-AL,P20,US-AL,Q264
...,...,...,...,...
107,US-AS,P4,Q16641,Q265
108,US-MP,description,American-dependent insular area in the western...,Q265
109,US-MP,P4,Q16644,Q265
110,US-DC,description,federal district of the United States of Ameri...,Q265


# Check for Existing Records in EEW KB

In [8]:
# Try primary identifier
query_iso3166_2 = """
PREFIX wd: <https://eew-edgi.wikibase.cloud/entity/>
PREFIX wdt: <https://eew-edgi.wikibase.cloud/prop/direct/>

SELECT ?item ?itemLabel ?identifier
WHERE {
  ?item wdt:P20 ?identifier .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

items_by_iso3166_2 = eew.sparql_query(
    query=query_iso3166_2,
    output="dataframe"
)

items_by_iso3166_2['eew_qid'] = items_by_iso3166_2['item'].apply(lambda x: x.split('/')[-1])

main_identifer_merge = pd.merge(
    left=tiger_plus_wd,
    right=items_by_iso3166_2[["identifier","eew_qid"]],
    how="left",
    on="identifier"
)

In [9]:
# Split matched and unmatched records
main_identifier_matches = main_identifer_merge[main_identifer_merge.eew_qid.notnull()].reset_index(drop=True)
main_identifier_mismatches = main_identifer_merge[main_identifer_merge.eew_qid.isnull()].reset_index(drop=True)

In [10]:
# Try another identifier on country code for the US territories
main_identifier_mismatches['identifier'] = main_identifier_mismatches.identifier.apply(lambda x: x.split('-')[-1])

query_country_code = """
PREFIX wd: <https://eew-edgi.wikibase.cloud/entity/>
PREFIX wdt: <https://eew-edgi.wikibase.cloud/prop/direct/>

SELECT ?item ?itemLabel ?identifier
WHERE {
  ?item wdt:P18 ?identifier .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

items_by_country_code = eew.sparql_query(
    query=query_country_code,
    output="dataframe"
)

items_by_country_code['eew_qid'] = items_by_country_code['item'].apply(lambda x: x.split('/')[-1])

alt_identifer_merge = pd.merge(
    left=main_identifier_mismatches.drop(columns=["eew_qid"]),
    right=items_by_country_code[["identifier","eew_qid"]],
    how="left",
    on="identifier"
)

In [11]:
# Join the two sets of properties matched to EEW KB identifiers
claims_from_source = pd.concat([
    main_identifier_matches.drop(columns=["identifier"]),
    alt_identifer_merge.drop(columns=["identifier"])
]).reset_index(drop=True)

In [13]:
# Any mismatches
claims_from_source[claims_from_source.eew_qid.isnull()].empty

True

In [22]:
property_details = eew.properties(output='dataframe')
property_details['property'] = property_details.property.apply(lambda x: x.split('/')[-1])
property_details['property_type'] = property_details.property_type.apply(lambda x: x.split('#')[-1])

In [23]:
property_details

,property,propertyLabel,property_type
0,P1,instance of,WikibaseItem
1,P2,subclass of,WikibaseItem
2,P3,related wikidata property,ExternalId
3,P4,related wikidata item,ExternalId
4,P5,formatter URL,String
5,P6,data source,WikibaseItem
6,P7,data format,WikibaseItem
7,P8,item of this property,WikibaseItem
8,P9,equivalent property,Url
9,P10,reference URL,Url


# Processing Workflow - Additional Claims

In [35]:
merge_props = ["property","value"]

for qid, source_props in claims_from_source[~claims_from_source.property.isin(['label','description'])].groupby("eew_qid"):
    existing_props = eew.qid_property_fetcher([qid])
    
    merged_props = pd.merge(
        left=source_props[["property","value","reference_qid"]],
        right=existing_props[merge_props],
        how="left",
        left_on=merge_props,
        right_on=merge_props,
        indicator='new'
    )
    
    new_props = merged_props[merged_props.new != 'both']
    if not new_props.empty:
        new_props_typed = pd.merge(
            left=new_props,
            right=property_details,
            how="left",
            on="property"
        )
        
        item = eew.wbi.item.get(qid)
        
        for index, row in new_props_typed.iterrows():
            refs = models.References()
            ref = datatypes.Item(
                prop_nr=properties['data source'],
                value=row['reference_qid']
            )
            refs.add(ref)
            
            if row['property_type'] == 'WikibaseItem':
                item.claims.add(
                    datatypes.Item(
                        prop_nr=row['property'],
                        value=row['value'],
                        references=refs
                    )
                )
            elif row['property_type'] == 'ExternalId':
                item.claims.add(
                    datatypes.ExternalID(
                        prop_nr=row['property'],
                        value=row['value'],
                        references=refs
                    )
                )
            elif row['property_type'] == 'GlobeCoordinate':
                coords = row['value'].split(',')
                item.claims.add(
                    datatypes.GlobeCoordinate(
                        prop_nr=row['property'],
                        latitude=float(coords[0]),
                        longitude=float(coords[1]),
                        references=refs
                    )
                )
                
            item.write(summary='Added additional claims from TIGER data matched on one or more identifiers')


# Processing Workflow - New Items

In [ ]:
tiger_refs = models.References()
ref_tiger = datatypes.Item(
    prop_nr=properties['data source'],
    value=tiger_states_datasource.ds.split('/')[-1]
)
tiger_refs.add(ref_tiger)

wd_refs = models.References()
ref_wd = datatypes.Item(
    prop_nr=properties['data source'],
    value=wd_states_datasource.ds.split('/')[-1]
)
wd_refs.add(ref_wd)

for index, row in process_items.iterrows():
    print("PROCESSING:", row.label, row.eew_qid)
    
    if isinstance(row.eew_qid, str):
        item = eew.wbi.item.get(row.eew_qid)
    else:
        item = eew.wbi.item.new()
    
    item.labels.set('en', row.label)
    item.descriptions.set('en', row.stDescription)

    # Instance of classification
    item.claims.add(
        datatypes.ExternalID(
            prop_nr=properties['instance of'],
            value=row.instance_of
        )
    )
    
    # FIPS 5-2 Alpha Code
    item.claims.add(
        datatypes.ExternalID(
            prop_nr=properties['FIPS 5-2 alpha'],
            value=row.fips5_2alpha,
            references=tiger_refs
        )
    )

    # FIPS 5-2 Numeric Code
    item.claims.add(
        datatypes.ExternalID(
            prop_nr=properties['FIPS 5-2 numeric'],
            value=row.fips5_2num,
            references=tiger_refs
        )
    )

    # FIPS 10-4 Code
    item.claims.add(
        datatypes.ExternalID(
            prop_nr=properties['FIPS 10-4'],
            value=row.fips10_4,
            references=tiger_refs
        )
    )
        
    # ISO 3166-2 Code
    item.claims.add(
        datatypes.ExternalID(
            prop_nr=properties['ISO 3166-2 code'],
            value=row.iso3166_2,
            references=tiger_refs
        )
    )

    # GNIS ID
    item.claims.add(
        datatypes.ExternalID(
            prop_nr=properties['GNIS ID'],
            value=row.gnis_id,
            references=tiger_refs
        )
    )
    
    # Coordinate location
    item.claims.add(
        datatypes.GlobeCoordinate(
            prop_nr=properties['coordinate location'],
            latitude=row.latitude,
            longitude=row.longitude,
            references=tiger_refs
        )
    )
        
    # Wikidata ID
    item.claims.add(
        datatypes.ExternalID(
            prop_nr=properties['related wikidata item'],
            value=row.wd_qid,
            references=wd_refs
        )
    )
    
    item.write(summary="Added new record from U.S. Census Bureau TIGER data linked to Wikidata ID")
        